In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    model_path: Path
    updated_model_path: Path
    updated_model_json_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories, save_model, save_object

2023-08-03 12:50:14.651438: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 12:50:14.717039: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 12:50:14.718159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 12:50:15.899786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,           
            model_path = config.model_path,
            updated_model_path = config.updated_model_path,
            updated_model_json_path = config.updated_model_json_path,
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES

        )

        return prepare_base_model_config

In [6]:
import sys
from cnnClassifier.exception import CustomException
import keras


In [7]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    # In case, vgg16 is about to used as the base model
    # NOTE: input_shape must have three channels
    def get_base_model(self):
        self.model = keras.applications.vgg16.VGG16(
            #input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )

        #save_object(path=self.config.model_path, obj= self.model, h5=True)

    @staticmethod
    def _prepare_full_model(base_model, classes, freeze_all, freeze_till, learning_rate, input_shape=None):
        if base_model is not None:        
            if freeze_all:
                for layer in base_model.layers:
                    base_model.trainable = False
            elif (freeze_till is not None) and (freeze_till > 0):
                for layer in base_model.layers[:-freeze_till]:
                    base_model.trainable = False            

        
            flatten_in = keras.layers.Flatten()(base_model.output)
            prediction = keras.layers.Dense(
                units=classes,
                activation='softmax'
            )(flatten_in)

            model = keras.models.Model(
                inputs=base_model.input,
                outputs=prediction
            )
            
        
        else:
            assert input_shape is not None, " WARNING: Input shape mus be provided!"
            model = keras.models.Sequential()
            model.add(keras.layers.Convolution2D(filters=64, kernel_size=(3, 3), padding='same', name='image_array', input_shape= input_shape))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation('relu'))
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
            model.add(keras.layers.Dropout(0.25))
            
            
            model.add(keras.layers.Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
            model.add(keras.layers.BatchNormalization())    
            model.add(keras.layers.Activation('relu'))        
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
            model.add(keras.layers.Dropout(0.25))

            model.add(keras.layers.Convolution2D(filters=256, kernel_size=(3, 3), padding='same'))
            model.add(keras.layers.BatchNormalization())           
            model.add(keras.layers.Activation('relu'))
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
            model.add(keras.layers.Dropout(0.25))

            model.add(keras.layers.Convolution2D(filters=512, kernel_size=(3, 3), padding='same'))
            model.add(keras.layers.BatchNormalization())          
            model.add(keras.layers.Activation('relu'))
            model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
            model.add(keras.layers.Dropout(0.25))

            model.add(keras.layers.Flatten())
            model.add(keras.layers.Dense(256))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation('relu'))
            model.add(keras.layers.Dropout(0.25))

            model.add(keras.layers.Dense(512))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation('relu'))
            model.add(keras.layers.Dropout(0.25))

            model.add(keras.layers.Dense(classes, 'softmax', name='predictions'))
        
        
        full_model = model            

        full_model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss = keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
                           )
        
        full_model.summary()
        return full_model
    
    def update_model(self):
        self.full_model = self._prepare_full_model(
            base_model = None, #self.model,
            classes=self.config.params_classes,
            freeze_all=False,
            freeze_till= None,
            learning_rate = self.config.params_learning_rate,
            input_shape = self.config.params_image_size

        )

        save_model(h5_path= self.config.updated_model_path, 
                   json_path= self.config.updated_model_json_path, 
                   model=self.full_model)

        

In [8]:
try:
    config = configurationManeger()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_model()

except Exception as e:
    raise CustomException(e, sys)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_array (Conv2D)        (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 128)       7